In [1]:
from PyQt5.QtWidgets import *
from PyQt5.QtCore import *
from PyQt5 import QtWidgets, uic
from PyQt5.QtGui import *
import sys
import os
from PIL import Image
import os, glob, numpy as np 
from keras.models import load_model
import socket, threading
basic_ui = uic.loadUiType("python_cnn_UI.ui")[0]

Using TensorFlow backend.


In [2]:
#버튼 출력 윈도우창   

class WindowClass(QMainWindow, basic_ui):
    
    # 데이터셋 저장한 디렉토리
    base_dir = '../KWGuideImg/dataset/train'
    categories=os.listdir(base_dir) #["기념","화도","옥의","비마","새빛","참빛","한울","연구"]
    
    def __init__(self):
        super().__init__()
        self.setupUi(self)
       
        self.cnn_result="" #이미지 분류 결과
        self.th = QThread() 
        self.serverbtn1.clicked.connect(self.Server1)
        self.serverbtn2.clicked.connect(self.Server2)
        self.show()
    
    def binder(self,client_socket, addr):  

        print('Connected by', addr); 
        try:
        
            data = client_socket.recv(4)  

            length = int.from_bytes(data, "big")    

            data = client_socket.recv(length) 

            msg = data.decode() 

            print('Received from', addr, msg) 

            global MSG 
            MSG = msg
            print("MSG : ",MSG)

            data = msg.encode() 

            length = len(data)      

            client_socket.sendall(length.to_bytes(4, byteorder='big')) 
            client_socket.sendall(data) 
           
        except:

            print("except : " , addr)  
                
        finally:

            client_socket.close() 
    
    #C#에서 파이썬으로 소켓통신을 통해 이미지 전송
    def Server1(self):
        
        server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM) 

        server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1) 

        server_socket.bind(('', 9999))

        server_socket.listen() 
        
        print("client 접속중...")
        
        
        try:
 
            client_socket, addr = server_socket.accept() 
            self.th = threading.Thread(target=self.binder, args = (client_socket,addr))  
            print("스레드 시작")
            self.th.start()
            
        except:
            print("server1 연결끊김")
            server_socket.close() 
        finally:

            server_socket.close() 
    
    def binder2(self,client_socket, addr): 
    
        print('Connected by', addr)
        try:
       
            data = client_socket.recv(4)  # socket의 recv함수는 연결된 소켓으로부터 데이터를 받을 대기하는 함수

            length = int.from_bytes(data, "big")    

            data = client_socket.recv(length) # 다시 데이터 수신

            msg = data.decode() # 수신된 데이터를 str형식으로 decode한다.

            print('Received from', addr, msg) 

            msg=self.cnn_result

            data = msg.encode() # 바이너리(byte)형식으로 변환한다.

            length = len(data)      # 바이너리의 데이터 사이즈를 구한다.

            client_socket.sendall(length.to_bytes(4, byteorder='big'))  

            client_socket.sendall(data) 
            
        except:

            print("except : " , addr)  # 접속이 끊기면 except가 발생

        finally:

            client_socket.close() # 접속이 끊기면 socket 닫음
        
    #훈련된 모델을 사용하여 이미지 예측 후, 예측 결과를 소켓통신하여 파이썬에서 C#으로 보냄
    def Server2(self):
        
        print(MSG)
        img = Image.open(MSG)
        img = img.convert("RGB")
        image_w = 64
        image_h = 64 #크기지정
        img = img.resize((image_w, image_h)) 
        data = np.asarray(img)
        X=np.array(data)

        X = X.astype("float") / 256
        X = X.reshape(-1, 64, 64,3)

        model=load_model('Building_cnn.h5')
        predictions = model.predict(X)  #X이미지 레이블 예측
        predictions[0]
        result = np.argmax(predictions[0])    # 예측 값중 가장 높은 신뢰도를 가진 레이블
        base_dir = '../KWGuideImg/dataset/train'
        categories=os.listdir(base_dir) #["기념","화도","옥의","비마","새빛","참빛","한울","연구"]
        self.cnn_result = str(categories[result])       
        print(self.cnn_result)

        server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM) 

        server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1) 

        server_socket.bind(('', 9999))

        server_socket.listen() 

        print("client 접속중...")
        
        try:

            client_socket, addr = server_socket.accept() 
            th = threading.Thread(target=self.binder2, args = (client_socket,addr))  

            th.start()
        except:
            print("server2 연결끊김")
            server_socket.close() # 에러가 발생하면 서버 소켓을 닫는다
        finally:
            server_socket.close() # 에러가 발생하면 서버 소켓을 닫는다.
                           
app = QApplication([])
ex = WindowClass()
sys.exit(app.exec_())


client 접속중...
스레드 시작
Connected by ('127.0.0.1', 64245)
Received from ('127.0.0.1', 64245) C:\github\KW_GuideBook\KWGuideImg\dataset\test\test_img3.jpg
MSG :  C:\github\KW_GuideBook\KWGuideImg\dataset\test\test_img3.jpg
C:\github\KW_GuideBook\KWGuideImg\dataset\test\test_img3.jpg
hwado
client 접속중...
Connected by ('127.0.0.1', 64249)
Received from ('127.0.0.1', 64249) accept


SystemExit: 0

C:\Users\meid9\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
